In [1]:
##importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import plotly.graph_objects as go
import seaborn as sns
import itertools
from scipy import *
from scipy.signal import butter, filtfilt
from scipy.interpolate import interp1d
from scipy.interpolate import CubicSpline
from scipy.interpolate import lagrange
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from statsmodels.nonparametric.kernel_regression import KernelReg
from IPython.display import display
from collections import defaultdict
import datetime
import math
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.font_manager as font_manager
from matplotlib import rc
import shutil
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm, LinearSegmentedColormap
from matplotlib.lines import Line2D

In [96]:
import requests
from bs4 import BeautifulSoup

In [116]:
import json

In [2]:
df = pd.read_csv("ad.csv", header=None)

In [9]:
df

,ID,fdr_pval,pval,zval,Gene_Name
0,CALM1,6.78E-06,6.87E-10,6.169166226,calmodulin 1
1,NSUN6,6.78E-06,1.93E-09,-6.003483316,NOP2/Sun RNA methyltransferase family member 6
2,PSMB7,6.78E-06,2.15E-09,5.985985897,proteasome subunit beta 7
3,TEX10,6.78E-06,1.89E-09,-6.006881244,testis expressed 10
4,ADORA2A /// SPECC1L-ADORA2A,8.67E-06,4.13E-09,-5.879031586,NaN
...,...,...,...,...,...
1063,GEM,0.049432244,0.004172339,-2.864829445,GTP binding protein overexpressed in skeletal ...
1064,TRO,0.049432244,0.004174226,2.86468624,trophinin
1065,EMX2,0.049653065,0.004200747,-2.862679916,empty spiracles homeobox 2
1066,PGP,0.049653065,0.00420051,2.862697765,phosphoglycolate phosphatase


In [4]:
row_slice = df.iloc[0, 0:5] 

In [20]:
row_slice

0           ID
1     fdr_pval
2         pval
3         zval
4    Gene_Name
Name: 0, dtype: object

In [6]:
df.columns = row_slice.values

In [8]:
df = df.drop(df.index[0]).reset_index(drop=True)

In [10]:
nan_indexes = df.index[df.iloc[:, 4].isna()].tolist()

In [11]:
nan_indexes

[4,
 5,
 7,
 17,
 20,
 27,
 28,
 41,
 46,
 51,
 57,
 59,
 61,
 66,
 82,
 83,
 97,
 110,
 114,
 128,
 133,
 148,
 149,
 162,
 190,
 201,
 202,
 205,
 214,
 215,
 224,
 230,
 235,
 242,
 252,
 264,
 269,
 272,
 293,
 301,
 312,
 315,
 320,
 334,
 351,
 354,
 356,
 371,
 388,
 393,
 428,
 436,
 458,
 464,
 468,
 507,
 518,
 531,
 537,
 553,
 563,
 583,
 593,
 605,
 612,
 659,
 666,
 670,
 702,
 710,
 742,
 743,
 758,
 762,
 773,
 783,
 786,
 794,
 805,
 819,
 839,
 856,
 857,
 860,
 870,
 872,
 873,
 885,
 891,
 920,
 921,
 931,
 939,
 940,
 947,
 949,
 964,
 973,
 974,
 982,
 992,
 997,
 1001,
 1020,
 1031,
 1034,
 1035,
 1044,
 1046]

In [12]:
first_column_values = df.iloc[nan_indexes, 0]

In [13]:
first_column_values

4                             ADORA2A /// SPECC1L-ADORA2A
5                       LOC101928747 /// RBMX /// SNORD61
7                  ATP6V1G2-DDX39B /// DDX39B /// SNORD84
17                                                   NRD1
20      IPW /// LOC101930404 /// PWARSN /// SNORD107 /...
                              ...                        
1031                                             AK021933
1034    MIR4315-1 /// MIR4315-2 /// MIR6080 /// PLEKHM...
1035                                             AK021537
1044                      HSPA8 /// SNORD14C /// SNORD14D
1046                                             KIAA0141
Name: ID, Length: 109, dtype: object

In [14]:
df_ad = df.drop(nan_indexes).reset_index(drop=True)

In [15]:
df_ad

,ID,fdr_pval,pval,zval,Gene_Name
0,CALM1,6.78E-06,6.87E-10,6.169166226,calmodulin 1
1,NSUN6,6.78E-06,1.93E-09,-6.003483316,NOP2/Sun RNA methyltransferase family member 6
2,PSMB7,6.78E-06,2.15E-09,5.985985897,proteasome subunit beta 7
3,TEX10,6.78E-06,1.89E-09,-6.006881244,testis expressed 10
4,ANP32B,8.73E-06,7.17E-09,-5.786889258,acidic nuclear phosphoprotein 32 family member B
...,...,...,...,...,...
954,GEM,0.049432244,0.004172339,-2.864829445,GTP binding protein overexpressed in skeletal ...
955,TRO,0.049432244,0.004174226,2.86468624,trophinin
956,EMX2,0.049653065,0.004200747,-2.862679916,empty spiracles homeobox 2
957,PGP,0.049653065,0.00420051,2.862697765,phosphoglycolate phosphatase


In [120]:
ad_ID = df_ad['ID']

In [121]:
ad_ID

0       CALM1
1       NSUN6
2       PSMB7
3       TEX10
4      ANP32B
        ...  
954       GEM
955       TRO
956      EMX2
957       PGP
958     NFAT5
Name: ID, Length: 959, dtype: object

In [122]:
ad_ID = ad_ID.astype(str)

In [123]:
# Convert the Series to JSON
series_json = ad_ID.to_json()

# Save the JSON to a file
with open('series.json', 'w') as file:
    file.write(series_json)

In [117]:
for item in ad_ID:
    print(item)

CALM1
NSUN6
PSMB7
TEX10
ANP32B
KCNE4
UGP2
ZNF415
MT1M
PRDX2
SLC25A46
CCNC
NOTCH2NL
UQCRC2
GOT1
SLC22A18AS
CXCR4
HSD17B7
RUFY3
PLSCR4
C3orf14
KIAA0100
CD200
CCK
PIK3CB
ATP9A
ANKRD28
LRP4
DDIT4
ZNF721
PDLIM1
KIZ
PXDC1
CMAS
FAM168A
SCARF1
SCG2
TUBB
ENGASE
HLA-DRB4
TALDO1
ZNF204P
EEF1A2
TIMM8A
CNTD2
SST
AEBP1
SFXN3
UQCRFS1
HEY2
TAOK2
CALB1
UBL3
APPL1
MTPAP
PYCR1
RAB13
AFG3L2
STAT4
DNTTIP2
NGRN
EPM2AIP1
IL10RA
PCDH12
RNF41
SEPHS1
GRAMD1C
CDS2
ID3
RPL19
SNX3
ACAD10
BAZ1A
NDRG4
TGFBR3
ASCC3
PKM
PSMG2
MRPL46
P2RY6
LMO4
FBXW4P1
P4HTM
ZSWIM1
FOXO4
ADD1
RHBDD3
INA
FHL2
PRMT2
CARTPT
DDX56
GABBR2
PFKFB3
MPP1
CROCCP2
FAM169A
PKD1P1
SLC38A7
MT2A
KBTBD11
RTN1
SLC25A4
PLXNB1
GOT2
PCYOX1L
VEZF1
CD2BP2
PSTPIP1
OSTF1
NMB
MICALL2
RASL12
EMC10
UCHL1
PDLIM4
EGR2
MUC6
TCIRG1
FGF12
SERPINI2
PPM1F
WDR82
PTPN3
MRPL15
SIRT3
FOXD1
MYOM1
OSBPL11
UBN1
RNPEPL1
PNOC
MLLT11
NCOR2
DVL1
CXCL1
EFHD1
GOLGA8A
TRIL
CDIP1
PPP1R16B
TSC1
ZCCHC24
CRYM
HTR2A
MRPS17
TBX6
ARGLU1
PPP1CB
ATP6V1G2
GTF3C1
MFHAS1
RRAGA
MTSS1
MZT2B
LPAR4

In [ ]:
##missing_values = df.isna().any()
##missing_values
##column_slice = df.iloc[1:6659, 0]
##df_ad.columns = row_slice.values
##df_ad.index = column_slice.values
##unique_count = column_slice.nunique()

In [18]:
df2 = pd.read_csv("pd.csv", header=None)

In [24]:
df2

,ID,fdr_pval,pval,zval,Gene_Name
0,PTGDS,0.000300333,1.99199168271491e-08,-5.612695314,prostaglandin D2 synthase
1,PPP4C,0.00112651,1.49434222107558e-07,-5.253255237,protein phosphatase 4 catalytic subunit
2,KIR2DL1,0.001247078,2.48141752035025e-07,-5.159098494,"killer cell immunoglobulin like receptor, two ..."
3,KIAA0319L,0.001373528,3.64403470731745e-07,-5.086670841,KIAA0319 like
4,PLOD1,0.003423695,1.36248407112127e-06,-4.830415531,"procollagen-lysine,2-oxoglutarate 5-dioxygenase 1"
...,...,...,...,...,...
231,ZMAT5,0.047978176,0.000723141,-3.380651718,zinc finger matrin-type 5
232,PILRA,0.048383807,0.000747723,-3.371454646,paired immunoglobin like type 2 receptor alpha
233,PLAGL2,0.048737711,0.000756425,-3.368266172,PLAG1 like zinc finger 2
234,NPLOC4,0.049009149,0.000766495,-3.364618741,"NPL4 homolog, ubiquitin recognition factor"


In [21]:
df2.columns = row_slice.values

In [23]:
df2 = df2.drop(df2.index[0]).reset_index(drop=True)

In [25]:
nan_indexes2 = df2.index[df2.iloc[:, 4].isna()].tolist()

In [26]:
nan_indexes2

[39, 40, 51, 113, 121, 123, 142, 177, 194]

In [27]:
first_column_values2 = df2.iloc[nan_indexes2, 0]

In [28]:
first_column_values2

39         CD97
40        GPR97
51     C17orf62
113        EMR2
121       SQRDL
123    FLJ35934
142       PRR24
177       LPPR2
194     C5orf55
Name: ID, dtype: object

In [29]:
df_pd = df2.drop(nan_indexes2).reset_index(drop=True)

In [30]:
df_pd

,ID,fdr_pval,pval,zval,Gene_Name
0,PTGDS,0.000300333,1.99199168271491e-08,-5.612695314,prostaglandin D2 synthase
1,PPP4C,0.00112651,1.49434222107558e-07,-5.253255237,protein phosphatase 4 catalytic subunit
2,KIR2DL1,0.001247078,2.48141752035025e-07,-5.159098494,"killer cell immunoglobulin like receptor, two ..."
3,KIAA0319L,0.001373528,3.64403470731745e-07,-5.086670841,KIAA0319 like
4,PLOD1,0.003423695,1.36248407112127e-06,-4.830415531,"procollagen-lysine,2-oxoglutarate 5-dioxygenase 1"
...,...,...,...,...,...
222,ZMAT5,0.047978176,0.000723141,-3.380651718,zinc finger matrin-type 5
223,PILRA,0.048383807,0.000747723,-3.371454646,paired immunoglobin like type 2 receptor alpha
224,PLAGL2,0.048737711,0.000756425,-3.368266172,PLAG1 like zinc finger 2
225,NPLOC4,0.049009149,0.000766495,-3.364618741,"NPL4 homolog, ubiquitin recognition factor"


In [59]:
common = df_ad[df_ad['ID'].isin(df_pd['ID'])]

In [67]:
common

,ID,fdr_pval,pval,zval,Gene_Name
748,AMPD2,0.035055575,0.002304869,-3.047847484,adenosine monophosphate deaminase 2
239,CEBPB,0.005885362,0.000130195,-3.826089967,CCAAT enhancer binding protein beta
880,CNN2,0.043461885,0.003377152,-2.931144872,calponin 2
580,DHRS7B,0.023581607,0.001207875,3.237013987,dehydrogenase/reductase 7B
491,FGR,0.018815185,0.0008235,-3.344772798,"FGR proto-oncogene, Src family tyrosine kinase"
108,PSTPIP1,0.00165097,1.69E-05,-4.302519111,proline-serine-threonine phosphatase interacti...
300,PTAFR,0.008673678,0.000236224,-3.676750486,platelet activating factor receptor
285,SPI1,0.007828284,0.000204211,-3.713749781,Spi-1 proto-oncogene
552,ST6GALNAC2,0.021932563,0.001074477,-3.270262485,"ST6 N-acetylgalactosaminide alpha-2,6-sialyltr..."
687,TLN1,0.030619365,0.00184979,-3.113345962,talin 1


In [66]:
common = common.sort_values(by='ID', ascending=True)

In [61]:
common2 = df_pd[df_pd['ID'].isin(df_ad['ID'])]

In [69]:
common2

,ID,fdr_pval,pval,zval,Gene_Name
67,AMPD2,0.023732051,0.000117013,-3.852301513,adenosine monophosphate deaminase 2
178,CEBPB,0.043977097,0.000548869,-3.455698265,CCAAT enhancer binding protein beta
173,CNN2,0.041499338,0.000500954,-3.480245913,calponin 2
196,DHRS7B,0.046093929,0.000629951,-3.418380067,dehydrogenase/reductase 7B
75,FGR,0.024032219,0.000129725,-3.826980476,"FGR proto-oncogene, Src family tyrosine kinase"
14,PSTPIP1,0.007305149,6.93049763844655e-06,-4.495811925,proline-serine-threonine phosphatase interacti...
89,PTAFR,0.025194987,0.000144932,-3.79959587,platelet activating factor receptor
32,SPI1,0.013586465,3.10311125562013e-05,-4.165764341,Spi-1 proto-oncogene
193,ST6GALNAC2,0.045898475,0.000624076,-3.420929286,"ST6 N-acetylgalactosaminide alpha-2,6-sialyltr..."
209,TLN1,0.046922001,0.000684041,-3.395896945,talin 1


In [68]:
common2 = common2.sort_values(by='ID', ascending=True)

In [78]:
column_names = ['ID', 'zval_AD', 'AD', 'zval_PD', 'PD']

In [71]:
df_common_deg = pd.merge(common, common2, on='ID', how='inner')

In [106]:
df_common_deg

,ID,zval_AD,AD,zval_PD,PD
0,PSTPIP1,-4.302519,down,-4.495812,down
1,CEBPB,-3.826090,down,-3.455698,down
2,WAS,-3.765404,down,-3.411374,down
3,TNFAIP2,-3.749207,down,-3.395644,down
4,SPI1,-3.713750,down,-4.165764,down
5,PTAFR,-3.676750,down,-3.799596,down
6,FGR,-3.344773,down,-3.826980,down
7,ST6GALNAC2,-3.270262,down,-3.420929,down
8,TNFRSF1B,-3.247623,down,-4.533312,down
9,TLN1,-3.113346,down,-3.395897,down


In [107]:
common_deg = df_common_deg['ID']

In [108]:
common_deg

0        PSTPIP1
1          CEBPB
2            WAS
3        TNFAIP2
4           SPI1
5          PTAFR
6            FGR
7     ST6GALNAC2
8       TNFRSF1B
9           TLN1
10         AMPD2
11          CNN2
12          VASP
13        DHRS7B
Name: ID, dtype: object

In [73]:
indices_to_drop = [1, 2, 4, 5, 6, 8]
columns_to_drop = [df_common_deg.columns[index] for index in indices_to_drop]
df_common_deg.drop(columns=columns_to_drop, inplace=True)

In [75]:
df_common_deg.insert(loc=df_common_deg.columns.get_loc('zval_x') + 1, column='new_col_1', value=np.nan)

In [76]:
df_common_deg['new_col_2'] = np.nan

In [79]:
df_common_deg.columns = column_names

In [82]:
df_common_deg['zval_AD'] = pd.to_numeric(df_common_deg['zval_AD'], errors='coerce')
df_common_deg['zval_PD'] = pd.to_numeric(df_common_deg['zval_PD'], errors='coerce')

In [86]:
df_common_deg['AD'] = df_common_deg['AD'].astype('object')
df_common_deg['PD'] = df_common_deg['PD'].astype('object')

In [87]:
for index, row in df_common_deg.iterrows():
    df_common_deg.at[index, 'AD'] = 'up' if row['zval_AD'] > 0 else 'down'
    df_common_deg.at[index, 'PD'] = 'up' if row['zval_PD'] > 0 else 'down'

In [89]:
df_common_deg = df_common_deg.sort_values(by='zval_AD', ascending=True)

In [91]:
df_common_deg.reset_index(drop=True, inplace=True)

In [ ]:
hsa-mir-1-3p
hsa-mir-16-5p
hsa-mir-124-3p
hsa-mir-34a-5p
hsa-let-7b-5p
hsa-mir-27a-3p
hsa-mir-155-5p
hsa-mir-129-2-3p
hsa-mir-103a-3p
hsa-mir-182-5p
hsa-mir-23b-3p
hsa-mir-107
hsa-mir-146a-5p
hsa-mir-147a
hsa-mir-374a-5p
hsa-mir-7-5p
hsa-mir-191-5p
hsa-mir-210-3p
hsa-mir-195-5p
hsa-mir-26a-5p

In [104]:
# List of miRNA names
mirna_names = [
    'hsa-mir-1-3p', 'hsa-mir-16-5p', 'hsa-mir-124-3p', 'hsa-mir-34a-5p', 'hsa-let-7b-5p',
    'hsa-mir-27a-3p', 'hsa-mir-155-5p', 'hsa-mir-129-2-3p', 'hsa-mir-103a-3p', 'hsa-mir-182-5p',
    'hsa-mir-23b-3p', 'hsa-mir-107', 'hsa-mir-146a-5p', 'hsa-mir-147a', 'hsa-mir-374a-5p',
    'hsa-mir-7-5p', 'hsa-mir-191-5p', 'hsa-mir-210-3p', 'hsa-mir-195-5p', 'hsa-mir-26a-5p'
]
# Dictionary to store dataframes with the miRNA names as keys
df_dict = {}

# miRDB search URL
search_url = 'https://mirdb.org/cgi-bin/search.cgi'

for mirna_name in mirna_names:
    # Form data for POST request
    form_data = {
        'species': 'Human',
        'searchBox': mirna_name,
        'searchType': 'miRNA'
    }
    
    # Send the POST request
    response = requests.post(search_url, data=form_data, verify=False)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse the response HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Find the table with the id 'table1'
        table = soup.find('table', id='table1')
        
        # Extract all table rows, skip the header if it's the first row
        rows = table.find_all('tr')[1:]
        
        # Extract Gene IDs from the 5th column (4th index) of each row
        gene_ids = [row.find_all('td')[4].get_text(strip=True) for row in rows]
        
        # Create a DataFrame and store it in the dictionary with the miRNA name as key
        df_dict[mirna_name] = pd.DataFrame(gene_ids, columns=['Gene ID'])
    else:
        print(f"Failed to retrieve data for {mirna_name}: {response.status_code} - {response.reason}")

/Users/admin/Desktop/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mirdb.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/admin/Desktop/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mirdb.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/admin/Desktop/anaconda3/lib/python3.11/site-packages/urllib3/connectionpool.py:1100: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mirdb.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/Users/admin/Desktop/anac

In [124]:
for miRNA_name, df in df_dict.items():
    print(f"DataFrame for {miRNA_name}:")
    print(df)
    print("\n" + "="*50 + "\n") 

DataFrame for hsa-mir-1-3p:
     Gene ID
0      HACD3
1        MMD
2    SLC44A1
3       PAX7
4      CDK14
..       ...
940  SPATA13
941  SLC28A3
942   SH2D4A
943    SATB2
944    ACSL6

[945 rows x 1 columns]


DataFrame for hsa-mir-16-5p:
     Gene ID
0      PAPPA
1       FASN
2      UNC80
3       FGF2
4     TNRC6B
...      ...
1411    RGS5
1412  OTUD6A
1413  KIF16B
1414   CENPN
1415  OGFOD3

[1416 rows x 1 columns]


DataFrame for hsa-mir-124-3p:
      Gene ID
0      OSBPL3
1     SLC50A1
2       ITGB1
3        SIX4
4      NFATC1
...       ...
1643    PDZD2
1644     SOD2
1645   GRIN3A
1646    OPRK1
1647  ADIPOR2

[1648 rows x 1 columns]


DataFrame for hsa-mir-34a-5p:
      Gene ID
0        HCN3
1      FAM76A
2        MDM4
3    RAP1GDS1
4     FAM167A
..        ...
894    LRRC55
895       PLN
896      PREB
897     TBRG1
898     CDH11

[899 rows x 1 columns]


DataFrame for hsa-let-7b-5p:
     Gene ID
0    STARD13
1      HMGA2
2     IGDCC3
3    IGF2BP1
4     FIGNL2
..       ...
985   SKI

In [125]:
%store df_dict

Stored 'df_dict' (dict)


In [109]:
# Initialize a dictionary to hold the resulting DataFrames with common elements
common_dfs = {}

for miRNA_name, df in df_dict.items():
    # Find common elements between the DataFrame's 'Gene ID' column and id_series
    common_genes = df['Gene ID'][df['Gene ID'].isin(common_deg)]
    
    # Create a new DataFrame from the common elements
    common_genes_df = pd.DataFrame(common_genes).reset_index(drop=True)
    
    # Create a new name by appending "_common" to the original miRNA name
    new_name = f"{miRNA_name}_common"
    
    # Store the resulting DataFrame in the common_dfs dictionary
    common_dfs[new_name] = common_genes_df


In [110]:
for df_name, df in common_dfs.items():
    print(f"DataFrame: {df_name}")
    print(df, "\n")

DataFrame: hsa-mir-1-3p_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-mir-16-5p_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-mir-124-3p_common
  Gene ID
0    TLN1 

DataFrame: hsa-mir-34a-5p_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-let-7b-5p_common
    Gene ID
0     PTAFR
1  TNFRSF1B 

DataFrame: hsa-mir-27a-3p_common
  Gene ID
0   PTAFR 

DataFrame: hsa-mir-155-5p_common
  Gene ID
0   CEBPB
1    SPI1 

DataFrame: hsa-mir-129-2-3p_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-mir-103a-3p_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-mir-182-5p_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-mir-23b-3p_common
  Gene ID
0    CNN2 

DataFrame: hsa-mir-107_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame: hsa-mir-146a-5p_common
  Gene ID
0     WAS 

DataFrame: hsa-mir-147a_common
Empty DataFrame
Columns: [Gene ID]
Index: [] 

DataFrame

In [113]:
filtered_common_dfs = {df_name: df for df_name, df in common_dfs.items() if not df.empty}

common_dfs = filtered_common_dfs

In [115]:
for df_name, df in common_dfs.items():
    print(f"DataFrame: {df_name}")
    print(df, "\n")

DataFrame: hsa-mir-124-3p_common
  Gene ID
0    TLN1 

DataFrame: hsa-let-7b-5p_common
    Gene ID
0     PTAFR
1  TNFRSF1B 

DataFrame: hsa-mir-27a-3p_common
  Gene ID
0   PTAFR 

DataFrame: hsa-mir-155-5p_common
  Gene ID
0   CEBPB
1    SPI1 

DataFrame: hsa-mir-23b-3p_common
  Gene ID
0    CNN2 

DataFrame: hsa-mir-146a-5p_common
  Gene ID
0     WAS 

DataFrame: hsa-mir-374a-5p_common
  Gene ID
0   CEBPB 

DataFrame: hsa-mir-7-5p_common
  Gene ID
0  DHRS7B 

DataFrame: hsa-mir-191-5p_common
  Gene ID
0   CEBPB 

